# MTS Generation with COSCI-GAN.


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from configs.Metric import Metric
from configs.SimulatedData import Proposed
from utils.metric import signature_on_batch, signature_metric
import mlflow
from sklearn.decomposition import PCA
from datetime import datetime
import io
from tensorflow.keras import layers


gpus = tf.config.list_physical_devices('GPU')

if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs.
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized.
    print(e)

# Load the Data.

In [ ]:
config= Proposed()

SEQUENCE_LENGTH = config.sequence_lenght_in_sample
GRANUARITY = config.granularity
OVERLAP = config.overlap
BS = config.batch_size
EPOCHS = config.epochs
NUM_SEQUENCE_TO_GENERATE = config.met_params.sequence_to_generate
STEP_LIMITATION = 1000
UPDATE_DISCRIMINATOR = 20

SIMULATED_DATA_PATH = "data/simulated_dataset/01 - Source Domain.h5"
SIMULATED_DATA_PATH2= "data/simulated_dataset/output_noise/0.25.h5"

N_SAMPLE_WIENER = SEQUENCE_LENGTH//4
FEAT_WIENER = 2
NOISE_DIM= (N_SAMPLE_WIENER, FEAT_WIENER)

LAMBDA_GLOBAL = 0.001
LAMBDA_LOCAL = 0.001

In [ ]:
def convert_dataframe_to_tensorflow_sequences(df:pd.DataFrame, sequence_lenght_in_sample, granularity, shift_between_sequences, batch_size, shuffle=True):
    sequence_lenght = int(sequence_lenght_in_sample*granularity)

    dset = tf.data.Dataset.from_tensor_slices(df.values)
    dset = dset.window(sequence_lenght , shift=shift_between_sequences, stride=granularity).flat_map(lambda x: x.batch(sequence_lenght_in_sample, drop_remainder=True))

    if shuffle:
        dset= dset.shuffle(256)

    dset = dset.batch(batch_size, drop_remainder=True)

    dset = dset.cache().prefetch(10)

    return dset

In [ ]:
df_simulated = pd.read_hdf(SIMULATED_DATA_PATH)
df_simulated = df_simulated.drop(columns='labels')

df_simulated2 = pd.read_hdf(SIMULATED_DATA_PATH2)
df_simulated2 = df_simulated2.drop(columns='labels')

dset_simulated = convert_dataframe_to_tensorflow_sequences(
    df_simulated, 
    SEQUENCE_LENGTH, 
    GRANUARITY, 
    int(OVERLAP* SEQUENCE_LENGTH),
    BS
)

dset_simulated2 = convert_dataframe_to_tensorflow_sequences(
    df_simulated2, 
    SEQUENCE_LENGTH, 
    GRANUARITY, 
    int(OVERLAP* SEQUENCE_LENGTH),
    BS
)

dset_simulated = dset_simulated.take(STEP_LIMITATION)
dset_simulated2 = dset_simulated2.take(STEP_LIMITATION)

## Verify if it is correct

In [ ]:
sequence = next(iter(dset_simulated))[0]
sequence2= next(iter(dset_simulated2))[0]

plt.figure(figsize=(18, 5))

ax = plt.subplot(211)
plt.title("Simulated Dataset 1.")
for i in range(sequence.shape[1]):
    plt.plot(sequence[:, i], label=f'feat {i+1}')
plt.grid()
plt.legend()

ax = plt.subplot(212)
ax.set_title("Simulated Dataset 2.")

for i in range(sequence.shape[1]):
    plt.plot(sequence2[:, i], label=f'feat {i+1}')
plt.grid()
plt.legend()

plt.show()

## Make some Wiener Noise.

In [ ]:
def wiener_process(batch:int, n_sample_wiener:int, n_feat_wiener:int):
    d_noise = tf.random.normal([batch, n_sample_wiener, n_feat_wiener])
    wiener_noise = tf.math.cumsum(d_noise, axis=1)
    return wiener_noise


seed = wiener_process(NUM_SEQUENCE_TO_GENERATE, N_SAMPLE_WIENER, FEAT_WIENER)


In [ ]:
def draw_arrow(A, B, color="b"):
    plt.arrow(A[0], A[1], B[0] - A[0], B[1] - A[1],
              length_includes_head=True, color=color)
    
def draw_arrows(xs, ys, color="b"):
    for i in range(xs.shape[0]-1):
        point0 = [xs[i], ys[i]]
        point1 = [xs[i+1], ys[i+1]]
        draw_arrow(point0, point1, color=color)

plt.figure(figsize=(18, 5))
plt.title("Example of the wiener process.")

draw_arrows(seed[0,:,0], seed[0,:,1], color="tab:blue")
plt.scatter(seed[0,:,0], seed[0,:,1], label='Wiener Process.', color='tab:blue')

plt.grid()
plt.legend()

## Make Model Architectures.

In [ ]:
# Define AdaIN Layers for Time Series
class AdaIN(tf.keras.layers.Layer):
    def __init__(self):
        super(AdaIN, self).__init__()

    def get_mean_std(self, x, eps=1e-5):
        _mean, _variance = tf.nn.moments(x, axes=[1], keepdims=True)
        standard_dev = tf.sqrt(_variance+ eps)
        return _mean, standard_dev

    def call(self, content_input, style_input):
        # print(content_input.shape, style_input.shape)
        content_mean, content_std = self.get_mean_std(content_input)
        style_mean, style_std = self.get_mean_std(style_input)
        adain_res =style_std* (content_input - content_mean) / content_std+ style_mean
        return adain_res

In [ ]:
def make_generator(n_sample_wiener:int, feat_wiener:int, style_vector_size:int):
    init = tf.keras.initializers.RandomNormal()

    content_input = tf.keras.Input((n_sample_wiener, feat_wiener), name="Content Input")
    style_input = tf.keras.Input((style_vector_size,), name="Style Input") 

    # Make a small projection...
    _content_input = tf.keras.layers.Flatten()(content_input)
    _content_input = tf.keras.layers.Dense(n_sample_wiener* feat_wiener, kernel_initializer=init)(_content_input)
    _content_input = tf.keras.layers.Reshape((n_sample_wiener, feat_wiener))(_content_input)

    # Make the style input 
    _style_input = tf.keras.layers.Dense(16, name='1')(style_input)
    _style_input = tf.keras.layers.Reshape((16, 1))(_style_input)

    x = AdaIN()(_content_input, _style_input)
    x = tf.keras.layers.Conv1DTranspose(256, 5, 2, padding='same', kernel_initializer=init)(x)
    x = tf.keras.layers.LeakyReLU()(x)

    x = tf.keras.layers.Conv1DTranspose(1, 5, 2, padding='same', kernel_initializer=init)(x)
    x = tf.keras.layers.LeakyReLU()(x)

    model = tf.keras.Model([content_input, style_input], x)
    return model


generator = make_generator(16, 2, 1)
generator.summary()
NOISE_DIM

In [ ]:
def make_global_discriminator(seq_length:int, n_feat:int, n_classes:int):
    _input = tf.keras.Input((seq_length, n_feat))

    x = tf.keras.layers.Conv1D(32, 5, 2, padding='same')(_input)
    x = layers.LeakyReLU()(x)

    x = layers.Flatten()(x)

    _output = layers.Dense(1, activation="sigmoid")(x)
    _class_output = layers.Dense(n_classes, activation="sigmoid")(x)

    model = tf.keras.Model(_input, [_output, _class_output])
    early_predictor = tf.keras.Model(_input, x, name="Local Discriminator")

    return model, early_predictor


def local_discriminator(seq_length:int, n_classes:int):
    _input = tf.keras.Input((seq_length, 1))

    x = tf.keras.layers.Conv1D(32, 5, 2, padding='same')(_input)
    x = layers.LeakyReLU()(x)

    x = tf.keras.layers.Conv1D(32, 5, 2, padding='same')(x)
    x = layers.LeakyReLU()(x)

    x = layers.Flatten()(x)
    _output = layers.Dense(1, activation="sigmoid")(x)
    _class_output = layers.Dense(n_classes, activation="sigmoid")(x)

    crit = tf.keras.Model(_input, _output, name="local discriminator")
    style_crit = tf.keras.Model(_input, _class_output, name="Style Discriminator")

    return crit, style_crit


def create_local_discriminators(n_discrs:int, sequence_length:int, n_style:int):
    _local_discr = []
    _style_disc = []

    for i in range(n_discrs):
        _crit, _s_crit = local_discriminator(sequence_length, n_style)

        _local_discr.append(_crit)
        _style_disc.append(_s_crit)

    return _local_discr, _style_disc 



In [ ]:
generators = [make_generator(16, 2, 1) for _ in range(df_simulated.shape[1])]

local_discriminators, style_discrs = create_local_discriminators(df_simulated.shape[1], SEQUENCE_LENGTH, 2)

global_discriminator, early_predictor = make_global_discriminator(SEQUENCE_LENGTH, df_simulated.shape[1], 2)

global_discriminator.summary()
local_discriminators[0].summary()
style_discrs[0].summary()

### Plot a Sequence.

In [ ]:
def generate(content_wp, style, training=True):
    style_labels = tf.zeros((NUM_SEQUENCE_TO_GENERATE, 1))+ style
    signals = np.array([g([content_wp, style_labels], training=training) for g in generators])
    signals = tf.transpose(signals, (1, 2, 0, 3))
    signals = tf.reshape(signals, signals.shape[:-1])
    return signals

def local_discrimination(sequences, training=True):
    crit = np.array([d(sequences[:, :, i], training=training) for i, d in enumerate(local_discriminators)])
    # crit = tf.transpose(crit, (1, 0, 2))
    return crit

In [ ]:
generated = generate(seed, 0.)

# after_training_generations
def plot_several_generations(generations:np.ndarray, nvertical:int=3, nhoriz:int=3):
    legend = [f"feat {j}" for j in range(generations.shape[-1])]

    plt.figure(figsize=(18, 10))
    plt.suptitle("Several Generations")

    for i in range(nvertical* nhoriz):
        ax = plt.subplot(nvertical, nhoriz, i+ 1)
        ax.set_title(f"sequence {i+1}")

        plt.plot(generations[i])
        ax.grid(True)
        plt.legend(legend)

    plt.tight_layout()
    plt.show()

plot_several_generations(generated) 


## Define losses

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def similarity_loss(extracted_features:np.ndarray):
    anchor = extracted_features[0]
    return tf.exp(-(tf.norm(extracted_features[1:]- anchor)))

In [ ]:
## Generate the reference signature.
real_sequence_style1 = next(iter(dset_simulated))
real_sequence_style2 = next(iter(dset_simulated2))

real_batch_signature_style1= signature_on_batch(real_sequence_style1, config.met_params.ins, config.met_params.outs, config.met_params.signature_length)
real_batch_signature_style2= signature_on_batch(real_sequence_style2, config.met_params.ins, config.met_params.outs, config.met_params.signature_length)



In [ ]:
generator_opt = [tf.keras.optimizers.RMSprop(1e-2) for _ in generators]
discriminator_opt = [tf.keras.optimizers.legacy.RMSprop(2e-3) for _ in local_discriminators]
global_discriminator_opt = tf.keras.optimizers.RMSprop(2e-3)

# generator_optimizer = tf.keras.optimizers.Adam(1e-4)
# discriminator_optimizer = tf.keras.optimizers.Adam(1e-6)    

## Define metrics 

In [ ]:
generator_metric = tf.keras.metrics.Mean()
local_discriminator_metric = tf.keras.metrics.Mean()
global_discriminator_metric = tf.keras.metrics.Mean()
similarity_metric = tf.keras.metrics.Mean()

correlation_metric_style1 = tf.keras.metrics.Mean()
correlation_metric_style2 = tf.keras.metrics.Mean()

## Tensorboard logs.

In [ ]:
date_str = datetime.now().strftime('%Y-%m-%d_%H_%M_%S')

BASE_DIR = f"logs - conditional COSCI-GAN/{date_str} - Conditional COSCI-GAN"
TRAIN_LOGS_DIR_PATH = f"{BASE_DIR}/fit"
GENERATION_LOG = f"{BASE_DIR}/Generations"

TRAIN_SUMMARY_WRITER = tf.summary.create_file_writer(TRAIN_LOGS_DIR_PATH)


def plot_to_buff(generation_s1:np.ndarray, generation_s2:np.ndarray, nvertical:int=3, nhoriz:int=3):
    legend = [f"feat {j}" for j in range(generation_s1.shape[-1])]

    fig = plt.figure(figsize=(18, 10))
    plt.suptitle("Generations After GAN Training.")

    for i in range(nvertical* nhoriz):
        current_style = i% 2
        ax = plt.subplot(nvertical, nhoriz, i+ 1)
        if current_style == 0:
            ax.set_title(f"sequence {i+1}. Style 1.")
            plt.plot(generation_s1[i])
        else: 
            ax.set_title(f"sequence {i+1}. Style 2.")
            plt.plot(generation_s2[i])
        ax.grid(True)
        plt.legend(legend)

    plt.tight_layout()

    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    plt.close(fig)
    return buf


def log_losses(epoch, plot_buf):
    image = tf.image.decode_png(plot_buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)

    with TRAIN_SUMMARY_WRITER.as_default():
        tf.summary.scalar("Generator Loss", generator_metric.result(), step=epoch)

        tf.summary.scalar("Local D loss", local_discriminator_metric.result(), step=epoch)
        tf.summary.scalar("Gobal D loss", global_discriminator_metric.result(), step=epoch)
        
        tf.summary.scalar("Mode Colapsing ?", similarity_metric.result(), step=epoch)
        tf.summary.scalar("Correlation Metric Style 1", correlation_metric_style1.result(), step=epoch)
        tf.summary.scalar("Correlation Metric Style 2", correlation_metric_style2.result(), step=epoch)

        tf.summary.image("Training Generations", image, step=epoch)


# Training Functions

In [ ]:
def generate_plots(noise, save_to):
    generated = generate(seed)

    fig =plt.figure(figsize=(18, 5))
    plt.title("Generation of the GAN during Training.")
    for i in range(generated.shape[-1]):
        plt.plot(generated[0, :, i], label=f'feat {i+1}')
    plt.grid()
    plt.legend()

    plt.savefig(save_to)
    plt.close(fig)

In [ ]:
bc = tf.keras.losses.BinaryCrossentropy()
error_classif = tf.keras.losses.SparseCategoricalCrossentropy()

def local_discriminator_loss(crits_on_real, crits_on_fake):
    individual_losses = []

    for i in range(crits_on_real.shape[0]):
        l1 = bc(tf.zeros_like(crits_on_real), crits_on_fake[i])
        l2 = bc(tf.ones_like(crits_on_real), crits_on_real[i])
        loss = (l1+ l2)/2
        individual_losses.append(loss)
        
    return individual_losses


def local_generator_loss(crit_on_fake):
    individual_losses = []
    true_label = tf.zeros(crit_on_fake[0].shape)

    for i in range(crit_on_fake.shape[0]):
        individual_losses.append(bc(true_label, crit_on_fake[i]))
        
    return np.array(individual_losses)

def style_classsification_loss(y_pred, y_true):
    return error_classif(y_true, y_pred)

def global_discriminator_loss(crit_on_real, crit_on_fake):
    l1 = bc(tf.zeros_like(crit_on_fake), crit_on_fake)
    l2 = bc(tf.ones_like(crit_on_real), crit_on_real)
    loss = (l1+ l2)/2

    return loss


def global_generator_loss(crit_on_fake):
    loss = bc(tf.ones_like(crit_on_fake), crit_on_fake)
    return loss

In [ ]:
def make_local_predictions(sequences, training=True):
    _crit_classif = []
    _crit_styles = []

    for i in range(len(local_discriminators)):
        crit, styles = local_discriminators[i](sequences[:,:, i], training=training)
        _crit_classif.append(crit)
        _crit_styles.append(styles)

    return np.array(_crit_classif), np.array(_crit_styles)

In [ ]:
# @tf.function
def discriminators_step(style1_ts, style2_ts, generators_, local_discriminators_, style_class_, global_discriminator_):
    noise= wiener_process(BS, N_SAMPLE_WIENER, FEAT_WIENER)
    style_labels = tf.zeros((BS,))

    with tf.GradientTape(persistent=True) as d_tape:
        style1_generated = [g([noise, style_labels], training=False) for g in generators_]
        style2_generated = [g([noise, style_labels+ 1.], training=False) for g in generators_]

        local_d1_gen = tf.convert_to_tensor([local_discriminators_[i](style1_generated[i], training=True) for i in range(len(local_discriminators_))])
        local_d2_gen = tf.convert_to_tensor([local_discriminators_[i](style2_generated[i], training=True) for i in range(len(local_discriminators_))])

        local_d1_real= tf.convert_to_tensor([local_discriminators_[i](style1_ts[:, :, i], training=True) for i in range(len(local_discriminators_))])
        local_d2_real= tf.convert_to_tensor([local_discriminators_[i](style2_ts[:, :, i], training=True) for i in range(len(local_discriminators_))])

        local_s1_real= tf.convert_to_tensor([style_class_[i](style1_ts[:, :, i], training=True) for i in range(len(style_class_))])
        local_s2_real= tf.convert_to_tensor([style_class_[i](style2_ts[:, :, i], training=True) for i in range(len(style_class_))])

        # Reshape the generated sequences for the global dicriminator.
        generated_reshaped1 = tf.stack(style1_generated, -1)
        generated_reshaped1 = tf.reshape(generated_reshaped1, (BS, SEQUENCE_LENGTH, generated_reshaped1.shape[-1]))

        generated_reshaped2 = tf.stack(style2_generated, -1)
        generated_reshaped2 = tf.reshape(generated_reshaped2, (BS, SEQUENCE_LENGTH, generated_reshaped2.shape[-1]))

        g_crit_gen1, _ = global_discriminator_(generated_reshaped1, training=True)
        g_crit_gen2, _ = global_discriminator_(generated_reshaped2, training=True)

        g_crit_real1, g_style_real1= global_discriminator_(style1_ts, training=True)
        g_crit_real2, g_style_real2= global_discriminator_(style2_ts, training=True)

        # (LOCAL DISCRIMINATOR) Real / Fake Loss 
        local_d_losses1 = [local_discriminator_loss(local_d1_gen[i], local_d1_real[i]) for i in range(len(local_discriminators_))]
        local_d_losses2 = [local_discriminator_loss(local_d2_gen[i], local_d2_real[i]) for i in range(len(local_discriminators_))]
        local_d_losses = local_d_losses1 + local_d_losses2

        #(LOCAL DISCRIMINATOR) Style CLassification:
        style_labels = tf.zeros((BS, 1))
        local_s1_loss = [style_classsification_loss(local_s1_real[i], style_labels) for i in range(len(local_discriminators_))]
        local_s2_loss = [style_classsification_loss(local_s2_real[i], style_labels+ 1.) for i in range(len(local_discriminators_))]
        local_style_loss = local_s1_loss + local_s2_loss

        # (GLOBAL DISCRIMINATOR) Real/ Fake Loss
        global_d_loss1 = global_discriminator_loss(g_crit_real1, g_crit_gen1)
        global_d_loss2 = global_discriminator_loss(g_crit_real2, g_crit_gen2)
        global_discrimination_loss = global_d_loss1+ global_d_loss2

        # (GLOBAL DISCRIMINATOR) Style Classification
        global_classif1 = style_classsification_loss(g_style_real1, style_labels)
        global_classif2 = style_classsification_loss(g_style_real2, style_labels+ 1.)
        global_style_loss = global_classif1+ global_classif2

    # (GOBAL DISCRIMINATOR): Real / Fake and style
    global_discr_gradient = d_tape.gradient([global_discrimination_loss, global_style_loss], global_discriminator_.trainable_variables)
    global_discriminator_opt.apply_gradients(zip(global_discr_gradient, global_discriminator_.trainable_variables)) 

    for i in range(len(local_discriminators_)): 
        grads = d_tape.gradient(local_d_losses[i], local_discriminators_[i].trainable_variables)
        discriminator_opt[i].apply_gradients(zip(grads, local_discriminators_[i].trainable_variables))

        grads = d_tape.gradient(local_style_loss[i], style_class_[i].trainable_variables)
        discriminator_opt[i].apply_gradients(zip(grads, style_class_[i].trainable_variables))

    local_discriminator_metric(tf.reduce_mean(local_d_losses))
    global_discriminator_metric(global_discrimination_loss)
        
# @tf.function
def generator_step(_generators, _local_discriminator, _style_discriminators, _global_discriminator):
    noise= wiener_process(BS, N_SAMPLE_WIENER, FEAT_WIENER)
    style_labels = tf.zeros((BS,))
    
    with tf.GradientTape(persistent=True) as g_tape:
        style1_generated = [g([noise, style_labels], training=True) for g in _generators]
        style2_generated = [g([noise, style_labels+ 1.], training=True) for g in _generators]

        # (LOCAL DISCRIMINATOR): Fakeness Predictions
        local_d1_gen = tf.convert_to_tensor([_local_discriminator[i](style1_generated[i], training=False) for i in range(len(_local_discriminator))])
        local_d2_gen = tf.convert_to_tensor([_local_discriminator[i](style2_generated[i], training=False) for i in range(len(_local_discriminator))])

        # (LOCAL DISCRIMINATOR): Style Predictions
        local_s1_gen = tf.convert_to_tensor([_style_discriminators[i](style1_generated[i], training=False) for i in range(len(_style_discriminators))])
        local_s2_gen = tf.convert_to_tensor([_style_discriminators[i](style2_generated[i], training=False) for i in range(len(_style_discriminators))])

        # Reshape the generated sequences for the global dicriminator.
        generated_reshaped1 = tf.stack(style1_generated, -1)
        generated_reshaped1 = tf.reshape(generated_reshaped1, (BS, SEQUENCE_LENGTH, generated_reshaped1.shape[-1]))

        generated_reshaped2 = tf.stack(style2_generated, -1)
        generated_reshaped2 = tf.reshape(generated_reshaped2, (BS, SEQUENCE_LENGTH, generated_reshaped2.shape[-1]))

        # (GLOBAL DISCRIMINATOR): Predictions.
        g_crit_gen1, g_style1 = _global_discriminator(generated_reshaped1, training=False)
        g_crit_gen2, g_style2 = _global_discriminator(generated_reshaped2, training=False)

        # (LOCAL DISCRIMINATOR): Real/ Fake.
        local_g_loss1 = local_generator_loss(local_d1_gen)
        local_g_loss2 = local_generator_loss(local_d2_gen)
        local_g_loss = local_g_loss1+ local_g_loss2

        # (LOCAL DISCRIMINATOR): Style.
        style_labels = tf.zeros((BS, 1))
        local_s1_loss = tf.convert_to_tensor([style_classsification_loss(local_s1_gen[i], style_labels) for i in range(len(_local_discriminator))])
        local_s2_loss = tf.convert_to_tensor([style_classsification_loss(local_s2_gen[i], style_labels+ 1.) for i in range(len(_local_discriminator))])
        local_style_preservation = local_s1_loss + local_s2_loss

        # (GLOBAL DISCRIMINATOR): Real / Fake.
        global_g_loss1= global_generator_loss(g_crit_gen1)
        global_g_loss2= global_generator_loss(g_crit_gen2)
        global_g_loss = global_g_loss1+ global_g_loss2

        # (GLOBAL DISCRIMINATOR): Style Classification.
        global_style1_preservation = style_classsification_loss(g_style1, style_labels)
        global_style2_preservation = style_classsification_loss(g_style2, style_labels+ 1.)
        global_style_preservation = global_style1_preservation + global_style2_preservation

        global_reduced = LAMBDA_GLOBAL* (global_g_loss+ global_style_preservation)
        g_loss = [LAMBDA_LOCAL* (l_loss + loc_sty_pre) + global_reduced for (l_loss, loc_sty_pre) in zip(local_g_loss, local_style_preservation)]
        
    # And Local Generators !
    for i in range(len(generators)):   
        grads = g_tape.gradient(g_loss[i], generators[i].trainable_variables)
        generator_opt[i].apply_gradients(zip(grads, generators[i].trainable_variables))

    generator_metric(tf.reduce_mean(local_g_loss))

In [ ]:
def train(dataset1, dataset2, epochs):
  losses = []
  total_steps = "?"
  for epoch in range(epochs):
    start = time.time()

    generator_metric.reset_states()
    global_discriminator_metric.reset_states()
    local_discriminator_metric.reset_states()

    for s, (sequence1, sequence2) in enumerate(zip(dataset1, dataset2)):
      if s % UPDATE_DISCRIMINATOR == 0:
        discriminators_step(sequence1, sequence2, generators, local_discriminators, style_discrs, global_discriminator)
        
      generator_step(generators, local_discriminators, style_discrs, global_discriminator)

      print(f"\r e {epoch}/{epochs}, s {s}/{total_steps}: Gen {generator_metric.result():0.4f}; Global discriminator: {global_discriminator_metric.result():0.4f}; Local discriminator: {local_discriminator_metric.result():0.4f}; Sim loss: {similarity_metric.result():0.4f}", end="")
      break
    if epoch == 0:
      total_steps = s

    stop = time.time()
    print()
    # print(f"\r[+] Epoch {epoch}/{epochs} in {(stop-start):0.4f} seconds. ({(stop-start)/total_steps:0.4f} s/step)")

    # generate_plots(seed, f"imgs/GAN_generations/{epoch}.png")
    # Make generations on seed
    seed_generation1 = generate(seed, 0., training=False)
    seed_generation2 = generate(seed, 1., training=False)
    buff = plot_to_buff(seed_generation1, seed_generation2)

    batch_signature_style1 = signature_on_batch(seed_generation1, [0, 1], [2, 3, 4, 5], config.met_params.signature_length)
    signature_difference1 = signature_metric(real_batch_signature_style1, batch_signature_style1)

    batch_signature_style2 = signature_on_batch(seed_generation2, [0, 1], [2, 3, 4, 5], config.met_params.signature_length)
    signature_difference2 = signature_metric(real_batch_signature_style2, batch_signature_style2)

    correlation_metric_style1(signature_difference1)
    correlation_metric_style2(signature_difference2)

    l = [generator_metric.result(), global_discriminator_metric.result(), local_discriminator_metric.result()]
    losses.append(l)
    log_losses(epoch, buff)

  return np.array(losses)

In [ ]:
training_losses = train(dset_simulated, dset_simulated2, EPOCHS)

## Plot Losses.

In [ ]:
plt.figure(figsize=(18, 5))
plt.title("Training Losses.")
plt.plot(training_losses[:, 0], ".-", label="Generator Loss")
plt.plot(training_losses[:, 1], ".-", label="Discriminator Loss")
plt.grid()
plt.legend()
plt.show()

# Let's Test it!

In [ ]:
after_training_generations = generator(seed, training=False)

plt.figure(figsize=(18, 5))
plt.title("Generation of the GAN whitout Training.")
for i in range(after_training_generations.shape[-1]):
    plt.plot(after_training_generations[0, :, i], label=f'feat {i+1}')
plt.grid()
plt.legend()
plt.show()

In [ ]:
after_training_decision = discriminator(after_training_generations)
after_training_decision[0]

## Test if Mode Colapsing

In [ ]:
# after_training_generations
def plot_several_generations(generations:np.ndarray, nvertical:int=3, nhoriz:int=3):

    legend = [f"feat {j}" for j in range(generations.shape[-1])]

    plt.figure(figsize=(18, 10))
    plt.suptitle("Generations After GAN Training.")

    for i in range(nvertical* nhoriz):
        ax = plt.subplot(nvertical, nhoriz, i+ 1)
        ax.set_title(f"sequence {i+1}")

        plt.plot(generations[i])
        ax.grid(True)
        plt.legend(legend)

    plt.tight_layout()
    plt.show()

plot_several_generations(after_training_generations)